# 44688-Data Analytics Capstone Project

## FDIP (Fire Department Incident Prediction)

##### 03/13/23 - 04/28/23

##### Debra D. "DeeDee" Walker

##### Northwest Missouri State University, Maryville MO 64468, USA

In [1]:
#Import the pandas framework to import and work with the dataset
import pandas as pd

#Import numpy
import numpy as np

#Import datetime so we can work with dates and times
import datetime as dt

#Original data was accessed through the csv file download due to the limitations of the API on the site for this large dataset. The API endpoint limits hits and requires data to be requested by page
# Data was pulled from https://data.cityofnewyork.us/Public-Safety/Incidents-Responded-to-by-Fire-Companies/tm6d-hbzd on March 16, 2023 then cleaned for use.
#Use the function read_csv from pandas and create a dataframe assigned to variable df
df = pd.read_csv('fdip_clean.csv', sep=',')

#convert INCIDENT_DATE_TIME from object to datetime format
df['INCIDENT_DATE_TIME'] = pd.to_datetime(df['INCIDENT_DATE_TIME'], infer_datetime_format = True)

df.head(n=5)

,IM_INCIDENT_KEY,INCIDENT_DATE_TIME,UNITS_ONSCENE,TOTAL_INCIDENT_DURATION,ZIP_CODE,BOROUGH_DESC,INCIDENT_CATEGORY,INCIDENT_CATNUM,LEVEL_CATEGORY,BOROUGH_NUM,Day_of_week,Hour_of_day,MONTH,DAY_NUM,INCIDENT_LENGTH,UNITS_CATEGORY
0,63583742,2018-07-21 12:18:19,1,0.66,10475,2 - Bronx,RESCUE & EMS,3,1,2,Saturday,12,7,6,30min-45min,1
1,63583743,2018-07-21 12:18:26,1,0.12,11230,4 - Brooklyn,RESCUE & EMS,3,1,4,Saturday,12,7,6,<=15min,1
2,63584267,2018-07-21 14:16:40,2,0.02,11204,4 - Brooklyn,HAZARDOUS CONDITION-NO FIRE,4,1,4,Saturday,14,7,6,<=15min,2
3,63584484,2018-07-21 15:07:51,1,0.37,11235,4 - Brooklyn,SERVICE CALL,5,1,4,Saturday,15,7,6,15min-30min,1
4,63584485,2018-07-21 15:08:36,1,0.32,11208,4 - Brooklyn,RESCUE & EMS,3,1,4,Saturday,15,7,6,15min-30min,1


##### Level Catergory is being dropped from the predictions because it is too skewed to the level of one to gain valid information and we can gain enough
#####  information from incident category, incident length, and units catergory

#### Split Data and extract features

In [2]:
from sklearn.model_selection import train_test_split

Xfeatures = df[['ZIP_CODE','Hour_of_day']]
ylabels = df[['INCIDENT_CATEGORY','INCIDENT_LENGTH','UNITS_CATEGORY']]
Xtrain, Xtest, ytrain, ytest = train_test_split(Xfeatures, ylabels, train_size=0.8, random_state=50)

#### Import sklearn classifiers, etc. for models

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

#### Random Forest with MultiOutput

In [4]:
forest_model = RandomForestClassifier(random_state=50, max_depth=20, n_estimators=200)
multi_forest = MultiOutputClassifier(forest_model)
multi_forest.fit(Xtrain,ytrain)
y_pred = multi_forest.predict(Xtrain)
print("Predicted y:", y_pred)
print()
print("Predicted probabilities:", multi_forest.predict_proba(Xtrain))
print()
print("The mean accuracy on the given test data and labels:", multi_forest.score(Xtrain,ytrain))

Predicted y: [['RESCUE & EMS' '<=15min' '1']
 ['RESCUE & EMS' '<=15min' '1']
 ['RESCUE & EMS' '<=15min' '1']
 ...
 ['RESCUE & EMS' '<=15min' '1']
 ['RESCUE & EMS' '<=15min' '1']
 ['RESCUE & EMS' '<=15min' '1']]

Predicted probabilities: [array([[7.77263226e-02, 1.62244483e-01, 7.35079689e-02, ...,
        5.00502669e-01, 9.97855190e-02, 0.00000000e+00],
       [6.94922300e-02, 1.48460336e-01, 6.43551235e-02, ...,
        5.61167802e-01, 7.82544930e-02, 1.33624928e-04],
       [7.68955308e-02, 1.42820816e-01, 8.19391434e-02, ...,
        5.05151762e-01, 8.44326841e-02, 1.55851020e-03],
       ...,
       [9.49329550e-02, 1.37825373e-01, 6.87216664e-02, ...,
        5.71712909e-01, 5.85591450e-02, 0.00000000e+00],
       [6.79825751e-02, 1.19948657e-01, 8.52473311e-02, ...,
        5.12611604e-01, 1.07033432e-01, 5.50364379e-04],
       [7.55942263e-02, 1.35746764e-01, 9.72057893e-02, ...,
        5.06706746e-01, 1.06638374e-01, 2.55910941e-06]]), array([[3.93343211e-02, 3.83534671e-01, 